# Introduction


In this challenge you will learn how to take an optimization problem and extract its mathematical formulation such that it can be solved with a quantum computer. Then, you will explore and implement in a guided way different types of algorithms suited to solve these optimization problems in the Noisy Intermediate-Scale Quantum (NISQ) era, where the quantum computers still have very limited capabilities. More specifically, in the Digital or Gate-based Quantum Computing model you will go through the process of building Variational Quantum Algorithms, while in the Analog or Adiabatic Quantum Computing paradigm the focus will be on Quantum Annealing. The idea of this challenge is to learn the basics of each topic presented and do small simulations with the different algorithms in order to compare them.


# 1. Problem formulation



Quantum computers are expected to bring an advantage against classical computers when solving different type of problems. In some cases, we can obtain "good" solutions with classical heuristic algorithms, but in order to ensure optimality we would require computations taking an amount of time (and memory in some cases) that grows exponentially with the size of the problem. A good example is the simulation of quantum systems or finding the ground state of quantum Hamiltonians, since only a quantum computer can reproduce efficiently their quantum properties.

<p style="text-align: center"><img src="https://i.imgur.com/TbGmXkE.png" width=700 /></p>

<p style="text-align: center"><em>Some examples of sectors where Quantum Computing could have a huge impact.</em></p>

However, another kind of problems that can be interesting to solve for scientific/academic purposes, but that also have a huge direct impact on industry, are the combinatorial optimization problems. These consist on finding the solution that minimizes a given function, among a set of finite possible solutions. While some of them can be efficiently solved with classical computers (we typically say that it can be efficiently solved if there's an algorithm that finds the solution in polynomial time), others are believed to be NP-Hard. These problems have applications in different fields such as logistics, energy market, scheduling, etc.


## QUBO formulation


We will focus on the combinatorials problems that can be mapped into a Quadratic Unconstraind Binary Optimization (QUBO) problem. These can be written as finding the solution $\vec{x}$ that minimizes the cost function

$$
f(\vec{x}) = \sum_i \sum_j Q_{ij} x_i x_j
$$

with $Q$ being the weights matrix and $x_i$ each one of the variables taking values $\{0, 1\}$ (therefore $x_i x_i = x_i$). As its own name indicates, we can only have terms that are at most quadratic with the variables and we cannot impose hard constraints (i.e. a priori any combination of values for the variables could be the optimal solution). However, since combinatorial problems usually require fulfilling some constraints, these will have to be introduced as soft constraints.

### Linear equality constraints

A linear constraint of the form

$$
\sum_i c_i x_i = M
$$

can be introduced as a penalty term

$$
\alpha \left(\sum_i c_i x_i - M \right)^2
$$

where $\alpha$ is the Lagrange multiplier that has to be tunned such that a solution breaking the constraint has a larger cost than any feasible solution. Note that if the term inside the parenthesis is not linear, when squaring it there could appear terms of order larger than quadratic.

### Linear inequality constraints

Another kind of constraint that can be implemented are linear inequality constraints of the form

$$
\sum_i c_i x_i \le M.
$$

In this case, the corresponding term is not so straightforward, and we will need to add some additional slack variables. These variables are used to convert the inequality into an equality as

$$
\sum_i c_i x_i = S
$$

where $S = \sum_i a_i s_i$, with $s_i$ being the new binary slack variables and $a_i$ are some fixed coefficients. The idea is that $S$ can take a range of values between the minimum achievable by the left hand side and $M$ by definition, and no constraints are introduced to the variables $s_i$. By doing this, if $\vec{x}$ forms a solution that fulfills the constraint, $\vec{s}$ will always find a value that makes the constraint term equal to 0, but the term will be $>0$ otherwise.


## Ising Hamiltonian



Prolems that can be written in QUBO form are of special interest since they can be mapped into an Ising Hamiltonian, which is one of the easiest to introduce in a quantum computer (a Hamiltonian is an operator that represents the energy of a system, modelizes the interactions between its elements, in our case the qubits, and dictaminates how their state will evolve over time). Ising Hamiltonians take the form

$$
H = \sum_{ij} J_{ij} \sigma^z_i \sigma^z_j
$$

where $\sigma^z_i$ corresponds to the Pauli operator acting on qubit $i$. Therefore, its eigenstates will be the set of $2^n$ states of the computational basis for a system of $n$ qubits, meaning that for our classical problems (where the solution corresponds to a single bitstring) the solution will always be on the eigenbasis.

The QUBO cost function and the Ising Hamiltonian share a very similar structure, the elements of a coefficients matrix are added with quadratic terms on the variables or operators. However, when doing the mapping from one to another, we have to be careful since the variables $x_i$ can take values 0 or 1, but the eigenvalues of the operators $\sigma_i$ are -1 and 1. This means that for unlike in the QUBO cost function where each term contributes either 0 or $Q_{ij}$, the Ising Hamiltonian terms will contribute a quantity that is either $-J_{ij}$ or $+J_{ij}$. In order to make this mapping a bit easier, it is common to change the basis of the Hamiltonian to the computational basis of 0's and 1's. This can be done by defining some new operators $b_{i}=\frac{I_2 - \sigma^z_i}{2}$ which have eigenvalues 0 and 1, such that the Hamiltonian can be built using directly the QUBO matrix $Q$.

Note that since each term acts on $n$ qubits (either with $\sigma_z/b_i$ or with an identity $I_2$), the Hamiltonian will result in a $2^n \times 2^n$ matrix. Since the scaling is exponential, we will be able to compute it for very small instances but it will become very inefficient as the size of the problem grows.


## Example: Traveling Salesperson Problem (TSP) formulation


As an example, we will present the Traveling Salesperson Problem and we will see how to construct the formulation from scratch. The problems consists of a set of $N$ locations that have to be visited sequentially exactly once, with a given cost for traveling from one location to another that is given by a matrix $W$. The goal is to find the path that visits all the locations (without repeating them) while minimizing the total traveling cost.

Different encodings can be used to describe the variables of the problem, but here we have selected the most simple one called one-hot encoding. We have to start by defining the variables of the problem, in this case any solution that fulfills the constraints of the problem can be described as a sequence of $N$ time steps, visiting a different location at each time step. The easiest way to be able to encode this is to define $N^2$ binary variables $x_{i, t}$ that indicate whether the location $i$ has been visited in the time step $t$ $(x_{i,t}=1)$ or not $(x_{i,t}=0)$.

<p style="text-align: center"><img src="https://i.imgur.com/7xP8snu.png" width=500 /></p>

<p style="text-align: center"><em>Example of the codification of the solution of the TSP problem with binary variables using the one-hot encoding. Each square represent a single variable being 1 or 0 if it is coloured or not.</em></p>

Now that we have the variables of the problem defined, we need to define the minimization term. It will correspond to the sum of the cost of traveling between each two consecutive locations:

$$
C_{min}(\vec{x}) = \sum_{t=1}^{N-1}\sum_{i=1}^N \sum_{j\ne i}^N W_{ij} x_{i, t} x_{j, t+1}
$$

In case some transition between two locations is not allowed, we can set a very large weight between them (equivalent to the value of the constraints Lagrange multiplier). This term will be minimized by the most optimal path among the set of all feasible paths, however there could be unfeasible solutions (solutions breaking the constraints) with a smaller cost. To avoid this, we have to add the constraints of the problem with Lagrange multipliers, the first one is that in each time step only one location is visited:

$$
C_1(\vec{x}) = \alpha_1 \sum_{t=1}^N \left(\sum_{i=1}^N x_{i, t} - 1\right)^2.
$$

The second constraint is that each locations is visited exactly once among all the time steps:

$$
C_2(\vec{x}) = \alpha_2 \sum_{i=1}^N \left(\sum_{t=1}^N x_{i, t} - 1\right)^2,
$$

with $\alpha_1, \alpha_2 > 0$, meaning that $C_1$ and $C_2$ will only be equal to 0 if the constraints are fulfilled. It is often interesting to find an upper bound for the minimum value needed for the Lagrange multipliers, in this case it can be easily seen that if we set their values to the maximum cost between two locations (i.e. the maximum value in $W$), any unfeasible solution will have a larger cost than any feasible solution. Therefore if the Lagrange multipliers are larger than this upper bound, we ensure the cost function to behave as expected.It is possible to look for better strategies or heuristics to find a lower value for this upper bound in case one is interested in lowering the values of $\alpha$.

The total cost function would be the sum of all the previous terms

$$
C(\vec{x}) = C_{min}(\vec{x}) + C_1 (\vec{x}) + C_2 (\vec{x})
$$

## Knapsack Problem (KP)

The Knapsack Problem is combinatorial problem with applications in logistics, finance and many other sectors where we are given a list of objects, each with an associated value $v_i$ and weight $w_i$. Assuming that we can only carry a maximum total weight $W$, we want to select the combination of items that maximizes the total value, without overpassing the maximum allowed weight.

## Links of interest

- [A. Lucas. "Ising formulations of many NP problems" (2014)](https://arxiv.org/abs/1302.5843)
- [F. Glover and G. Kochenberger and Y. Du. "A Tutorial on Formulating and Using QUBO Models" (2019)](https://arxiv.org/abs/1811.11538)

# Challenge
---

1) QUBO formulation for the Knapsack Problem (KP)

    1.1) Give the analytical description of the QUBO formulation for the Knapsack Problem

    1.2) Find the optimal number of slack variables and their coefficients $a_i$ for any range (assume that $S$ has a range $[0, M]$)

    1.3) Find the minimum value for the Lagrange multiplier in order to ensure the problem constraints.

    1.4) Write a code that given a KP instance, automatically generates the QUBO matrix as the weights matrix $Q$ from the equation above.

    1.5) Write a function that given a solution, returns the value for the slack variables that fulfill the constraint (if possible).

    1.6) Code a function that given a bitstring solution, returns the associated cost function value using the Hamiltonian matrix.

    Optional:

    1.7) Extend the formulation to include the following dependencies between the items accepted by the solution:

    - If items $i$ and $j$ are included, the total value is increased by an additional quantity.
        
    - If one of the items $i$ and $j$ is included without the other, the total value is decreased by a given quantity.

---

# Variational Quantum Algorithms (VQAs) for optimization problems



The most popular paradigm of quantum computing is the Digital or Gate-based approach, where the qubits start in a given state (typically $|0\rangle^{\otimes n}$) on top of which a circuit formed by quantum logical gates is applied. Then, the qubits are measured and a single bitsting is obtained. By repeating the execution and measurment with the same circuit multiple times, we can reconstruct the distribution representing the superposition on the final state.

This sound cool on paper and some very promising quantum algorithms have been proposed such as the Shor's algorithm (1994) for factorization or the Grover's algorithm (1996) for unstructured search. However, building the devices capable of running these circuits without errors is an extremely hard task. Some of the main problems that arise are scaling the number of qubits since in order to run many of the real use-cases we would need thousands or millions of logical qubits, the coherence time (the average time during which the qubits maintain their quantum properties without loosing their quantum information) which will determine how deep a circuit can be, or the noise that the qubits experience during the execution (error correction protocols will be needed, where a set of physical qubits are converted into a single logical qubit, thus greatly increasing the number of qubits and depth required).

In this context, some years ago appeared the idea of running Variational Quantum Algorithms for the so-called Noisy Intermediate-Scale Quantum (NISQ) devices (i.e. the quantum computers that we will have for the following years/decades). These are classical-quantum hybrid algorithms that try to take advantage of the limited quantum resources available, together with the powerful classical optimization algorithms that have been developed for a long time. They work by defining a parametrized quantum circuit that generates a quantum state called ansatz $|\psi(\vec{\theta})\rangle$ which is used to evaluate the cost function that we are willing to minimize. Starting from a given set of parameters, the cost function is evaluated and a classical optimizer proposes a new set of parameters that are tested iteratively until the algorithm converges or is stopped.

<p style="text-align: center"><img src="https://i.imgur.com/GJwNJeM.png" width=700 /></p>

<p style="text-align: center"><em>Variational Quantum Algorithm scheme. The workflow is repeated iteratively until we take the measured quantum state as the final solution.</em></p>

The advantage of using this workflow is that the quantum part is not so much sensitive to the noise and allows to use shallow circuits. On the other hand, this approach may require many iterations that increase the time and cost of running the algorithm and it is not guaranteed that it will find an optimal result. In fact, during the last years there has been a significant number of studies trying to face the problems of barren plateaus, local minima, ansatz expressivity, and scalability of the number of variational parameters.


## Ansatz


An ansatz is defined as the state generated by a variational quantum circuit, that is used as a guess to evaluate a cost function. It is very common to also use the term ansatz to refer to the quantum circuit that generates this trial quantum state. The selection of the ansatz is one of the most crucial parts of a VQA, many variational algorithms differ only on the design of the ansatz. Usually (but not always) they are divided into layers, and the number of layers $p$ is a free parameter that can be tuned depending on the resources that we are willing to spend to run the algorithm. An important concept to take into account when selecting the ansatz is its expressivity, depending on the gate structure and the number of layers our ansatz will be able to explore one part of the Hilbert space or another. If the optimal or "good" solutions of our problems are in a region that our ansatz cannot reach, we will encounter a strong limitation in our algorithm.

Also, at the beginning of the circuit we can often find a set of gates that act as state preparation to bring the initial state $|0\rangle^{\otimes n}$ into some desired state $|\psi_{init}\rangle$. In most cases, this is chosen to be a Hadamard gate acting on each one of the qubits in order to start at the state $|+\rangle$ (i.e. a flat superposition of all states of the computational basis) so that all the solutions start with the same probability and we already start with a state that can benefit from the properties of quantum superpositions.

One of the simplest ones is the so-called hardware-efficient ansatz, where each layer is divided into a set of single-qubit parametrized gates (e.g. RY($\theta_i$)) and a set of two-qubit gates (e.g. CNOT) between the qubits that we can make interact without the need of additional gates (e.g. in the case of superconducting qubits, it will correspond to the qubits that are physically connected to each other in the chip). This structure is copied for $p$ layers, each one with its own variational parameters.

<p style="text-align: center"><img src="https://i.imgur.com/dPVmUYn.png" width=800 /></p>

<p style="text-align: center"><em>On the left side we can see the topology of the chip, while on the right we have an example of a Hardware-efficient Ansatz. Parametrized Ry gates are taken as the single-qubit gates, and the CNOT gates are applied as two-qubit gates between the qubits physically connected.</em></p>

Unlike the hardware-efficient, there are other ansatzes that depend on the problem to be solved. This can be taken into account by introducing different relative phases depending on the quality of the solutions, designing ansatzes that generate superpositions which include only feasible solutions, etc. 


## Cost function


Usually the cost function to minimize in VQAs for classical problems won't have many limitations since one can always get the solution corresponding to the measured bitstring and compute any complex cost function. However, the most common approach is to encode solution of the optimization problem as the ground state of some Hamiltonian as seen in the first section. By doing this, the cost can be computed as the expected value of the final state with the problem Hamiltonian, written as

$$
C(\vec{\theta}) = \langle \psi (\vec{\theta})|H|\psi(\vec{\theta})\rangle
$$

Also another very interesting task to solve with quantum computers and VQAs is finding the ground state of quantum Hamiltonians (for which the eigenstates do not correspond to the computational basis), therefore this method is of special interest.

## Optimizer


As mentioned before, the variational parameters of VQAs are updated using classical optimizers. They work by recieving a function to be optimized which will be called at every iteration and an initial set of parameters, in our case the function that will be evaluated iteratively is the execution of the quantum circuit to estimate the final state which is then used to get the cost function value. Also, some optimizers require evaluating the function multiple times at each iteration, for example the gradient-based methods will need to check the function value in slightly different points of the parameters space before proposing a new set of parameters for the next iteration.

The selection of the optimizer is important since it determines the number of resources (both classical and quantum) that you will need to run your algorithm, and it will also determine to which final state it will converge.

The total number of quantum circuit calls in an optimization process is given by $n\_shots \cdot n\_iter \cdot n\_evals$, where $n\_shots$ is the number of times that a circuit is measured to recover the final state, $n\_iter$ is the number of iterations that the optimizer runs before convergence (or manual stop), and $n\_eval$ is the number of evaluations that the optimizer needs to do in each iteration.

## Links of interest

- First proposal of a VQA: [Peruzzo et Al. "A variational eigenvalue solver on a photonic quantum processor"](https://arxiv.org/abs/1304.3061)
- [M. Cerezo et Al. "Variational Quantum Algorithms" (2021)](https://arxiv.org/abs/2012.09265)
- [Huang et Al. "Near-term quantum computing techniques: Variational quantum algorithms, error mitigation, circuit compilation, benchmarking and classical simulation" (2023)](https://arxiv.org/abs/2211.08737)

# Challenge
---

2) Variational Quantum Eigensolver for the KP

    2.1) Define and code a hardware-efficient ansatz suitable for a line connectivity ($q_0-q_1-...-q_{n-1}$) with an arbitrary number of layers $p$.

    2.2) Write a function that given a set of parameters $\vec{\theta}$, an anstaz and a cost function, runs the circuit and evaluates the cost function by measuring $n\_shots$ times.

    2.3) Code the workflow of the entire variational algorithm where the circuit is executed and the parameters are updated iteratively using a Scipy optimizer (e.g. Powell, BFGS, Newton-CG, etc.)

    2.4) Run simulations with different number of ansatz layers to check the scalability of the algorithm.

    Optional:

    2.5) Try other type of optimizers (e.g. CMA-ES evolutionary optimizer)

    2.6) Look for a strategy to select the initial values of the variational parameters that improves the performance of the algorithm.

---


# Adiabatic Quantum Computing (AQC) and Quantum Annealing (QA)

When talking about quantum computing, most people think of the gate-based (or digital) model, that is a universal form of quantum computing in which most of the research so far has been done, but actually there are different other interesting proposals. The second most predominant one is the Adiabatic Quantum Computing (AQC) or Analog approach, that is also proven to be universal in its most general implementation.

<p style="text-align: center"><img src="https://i.imgur.com/W47VDxI.png" width=800 /></p>

<p style="text-align: center"><em>Brief comparison between the Digital and Analog models.</em></p>

The Analog model relies on the Adiabatic Theorem which states that a quantum system evolved under a time-dependente Hamiltonian $H(t)$, will stay on the same eigenstate during all the evolution if it is done slowly enough. In our case, the goal will be to start in the ground state of a simple Hamiltonian with an easy to prepare known ground state, also called mixer Hamiltonian $H_M$ (usually the mixer selected is $H_M=\sum_i^n \sigma^x_i$, with the ground state $|\psi\rangle = |+\rangle^{\otimes n}$), and continiously transition to the problem Hamiltonian $H_P$. At the end of the evolution we should find the ground state of $H_P$ encoding the solution to our problem. In order to determine how slow this evolution has to be made, in a first approximation the Adiabatic Theorem tells us that the total evolution time has to be grow as the inverse of the square of the minimum gap ($t_f \sim \frac{1}{\Delta^2}$, with $\Delta$ being the minimum value of the gap between the ground state and the first excited state during all the evolution, assuming that the ground state of $H_P$ is not degenerated). The smaller the gap, the slower the process will have to be.

The quantum state will be evolved following the time-dependent Schrödinger equation

$$
i\hbar \frac{d}{dt}|\psi(t)\rangle = \hat{H}|\psi(t)\rangle,
$$

with the time-dependent Hamiltonian being

$$
\hat{H}(t) = (1-s(t))\hat{H}_M + s(t) \hat{H}_P.
$$


$s(t)$ is known as the scheduling function, it has to be continuous between $t=0$ and $t=t_f$ and meet the boundary conditions $s(0)=0$ and $s(t_f)=1$. This imposes that at the start of the evolution we find the Hamiltonian $H_M$, and at the end we have $H_P$. In the most simple case, we will have a linear schedule of the form $s(t)=\frac{t}{t_f}$. This is the most simple Hamiltonian evolution for this purpose, but it may be interesting to include additional terms, called Counter-diabatic terms, that fade at the start and at the end of the evolution and which act as catalysts, allowing for a faster evolution without leaving the adiabatic regime.

Moreover, by designing more complex schedules we can reduce the required total time $t_f$ while still following the conditions imposed by the Adiabatic Theorem. For example one could design a scheduling function that evolves faster in the zones where the gap between the ground state and the first excited state is larger, and slows down as it approaches the minimum gap.

This algorithm is called Quantum Annealing (QA), but so far we only have seen it in the context of Adiabatic Quantum Computing where we try stay at the ground state at all times. In that case, in order to reach univsality it requires being able to apply more complex interactions than the ones appearing in our problem Hamiltonians. It is also possible to reach universality with QA by using diabatic techniques (i.e. running evolutions that use excited states).

Nevertheless, quantum annealers are specialy good for optimization problems, for which we don't need to achieve a universal device. Also, because of their nature and the slow tunning of the qubit interactions, these kind of quantum computers are less sensitive to the noise than gate-based devices, which means that they don't have to rely on quantum error correction protocols to provide competitive results.

## Links of interest

- [T. Albash and D. Lidar. "Adiabatic quantum computation" (2018)](https://arxiv.org/abs/1611.04471)
- [Guéry-Odelin et Al. "Shortcuts to adiabaticity: Concepts, methods, and applications" (2019)](https://arxiv.org/abs/1904.08448)

# Challenge
---

3) Quantum Annealing (QA)

    3.1) Construct the `qibo` problem Hamiltonian object from the QUBO matrix and the mixer Hamiltonian $H_M = \sum_i^n \sigma^x_i$

    3.2) Write a function that performs an annealing evolution with a linear schedule and evaluates the cost function with the final state

    3.3) Study how the total annealing time $T$ and the time step $dt$ affect the quantum state evolution and the quality of the final result

    3.4) Study how the value of the Lagrange multipliers affects the final result

    Optional:

    3.5) Define a quadratic scheduling function with free parameters and find the optimal coeffcients with a classical optimizer.

---


# Connection between AQC and DQC


It has been formally proven that Adiabatic Quantum Computing and Digital Quantum Computing are two paradigms that are equivalent with a polynomial overhead. Any quantum circuit in gate-based QC can be mapped to a target Hamiltonian to be solved by an adiabatic quantum computer and vice-versa. The direct implication is that if one of them is universal, the other will also have to be a universal form of quantum computing.


A typical adiabatic evolution from a mixing Hamiltonian $H_M$ to a problem Hamiltonian $H_P$ can be digitalized using the Trotter-Suzuki decomposition. The equivalence between both algorithms becomes stronger as the number of layers of the expansion tends to infinity $(p\rightarrow \infty)$, but as seen before, a circuit with these characteristics cannot be executed in a NISQ device.

Inspired by this idea, in 2014 a proposal of a new VQA was published where by taking a similar decomposition but with a much more reduced number of layers which included some variational parameters, it was possible to build an ansatz that was quite competitive for solving optimization problems.

## Links of interest

- [Aharonov et Al. "Adiabatic Quantum Computation is Equivalent to Standard Quantum Computation" (2005)](https://arxiv.org/abs/quant-ph/0405098)

# Challenge
---

4) From QA to VQAs

    4.1) Find online and implement the Variational Quantum Algorithm that is inspired by digitalized version of QA.

    4.2) Study the scaling and parameters selection as done in previous sections.

    4.3) Compare (qualitatively or quantitatively) this algorithm with the VQE and QA algorithms.

---

# Towards more complex algorithms


The algorithms explored in the previous section are some of the simplest for each paradigm. However, we can constantly find new publications proposing new ideas that may outperfom the state-of-the-art algorithms. In many cases, these are variations based on the algorithms seen here that try to improve some aspect such as quality of the solution reached, convergence speed, number of parameters needed, depth of the circuit, etc.

# Challenge
---

5) Towards more complex algorithms

    Explore and implement variations of the previous algorithms and compare them with the original ones. Choose the algorithm of your will, here you can find some suggestions:

    Analog:

    - [Reverse quantum annealing](https://arxiv.org/abs/1906.10889)

    - [Quantum annealing with counterdiabatic driving](https://arxiv.org/abs/1904.08448)

    Digital:

    - [Layer VQE (L-VQE)](https://arxiv.org/abs/2102.05566)

    - [Linear Ramp QAOA (LR-QAOA)](https://arxiv.org/abs/2405.09169)

    - [ADAPT-VQE](https://arxiv.org/abs/1812.11173)

---